# Spatial Joins Testing

For allowing the kernal for your conda environment to be found by Jupyter

https://stackoverflow.com/questions/39604271/conda-environments-not-showing-up-in-jupyter-notebook  
`python -m ipykernel install --user --name myenv --display-name "Python (myenv)"`

In [41]:
import pandas as pd
import geopandas as gpd
import requests, json
from shapely.geometry import Point

In [5]:
zip_url = 'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/oh_ohio_zip_codes_geo.min.json'

In [6]:
# Return the data
print('Getting Zip Code Shapefile Data...')
response = requests.get(zip_url)
zips = response.json()

Getting Zip Code Shapefile Data...


In [7]:
jobs = pd.read_csv('./data/jobs.csv')

In [9]:
jobs.head()

,id,status,noCharge,completedOn,invoice_total,lon,lat,zip
0,30500170,Hold,False,NaN,0.0,-82.966349,40.079448,43229
1,28460822,Completed,False,2019-07-19T19:15:00,0.0,-83.000895,40.019350,43202
2,29543583,Canceled,False,2019-07-19T14:45:22.7485338,0.0,-83.014422,40.053505,43214
3,31113730,Completed,False,2019-06-28T21:42:00,2273.0,NaN,NaN,43081
4,31264388,Completed,True,2019-06-26T13:36:00,0.0,-83.136459,40.154421,43017


## Convert Pandas Dataframe to a GeoDataFrame

In [45]:
# Attempt #1
gdf = gpd.GeoDataFrame(jobs, geometry=gpd.points_from_xy(jobs.lon, jobs.lat))
gdf.crs = {'init': 'epsg:4326'}

# Attempt #2
geom = jobs.apply(lambda x : Point([x['lon'],x['lat']]), axis=1)
gdf = gpd.GeoDataFrame(jobs, geometry=geom)
gdf.crs = {'init': 'epsg:4326'}
gdf.head()

,id,status,noCharge,completedOn,invoice_total,lon,lat,zip,geometry
0,30500170,Hold,False,NaN,0.0,-82.966349,40.079448,43229,POINT (-82.966 40.079)
1,28460822,Completed,False,2019-07-19T19:15:00,0.0,-83.000895,40.019350,43202,POINT (-83.001 40.019)
2,29543583,Canceled,False,2019-07-19T14:45:22.7485338,0.0,-83.014422,40.053505,43214,POINT (-83.014 40.054)
3,31113730,Completed,False,2019-06-28T21:42:00,2273.0,NaN,NaN,43081,POINT (nan nan)
4,31264388,Completed,True,2019-06-26T13:36:00,0.0,-83.136459,40.154421,43017,POINT (-83.136 40.154)


In [46]:
jobdf = gdf.dropna()

In [47]:
jobdf.head()

,id,status,noCharge,completedOn,invoice_total,lon,lat,zip,geometry
1,28460822,Completed,False,2019-07-19T19:15:00,0.0,-83.000895,40.019350,43202,POINT (-83.00089 40.01935)
2,29543583,Canceled,False,2019-07-19T14:45:22.7485338,0.0,-83.014422,40.053505,43214,POINT (-83.01442 40.05351)
4,31264388,Completed,True,2019-06-26T13:36:00,0.0,-83.136459,40.154421,43017,POINT (-83.13646 40.15442)
5,31052042,Completed,True,2019-07-17T14:53:00,0.0,-83.019600,40.040534,43214,POINT (-83.01960 40.04053)
6,31292052,Completed,False,2019-06-26T18:59:00,334.0,-82.803314,39.965785,43068,POINT (-82.80331 39.96578)


## Convert GeoJSON to GeoDataFrame

In [48]:
dfzips = gpd.read_file(zip_url) #Easy enough to just read from URL!

In [58]:
dfzips.crs = {'init': 'epsg:4326'}
dfzips.head()

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,39,45830,3945830,B5,G6350,S,245664720,465444,+40.9084596,-084.0959329,N,"MULTIPOLYGON (((-84.05180 40.84658, -84.05342 ..."
1,39,45877,3945877,B5,G6350,S,71903359,339959,+40.9591214,-083.9335780,N,"POLYGON ((-83.95902 40.91493, -83.95940 40.914..."
2,39,44859,3944859,B5,G6350,S,78526283,529089,+41.0219631,-082.3307243,N,"POLYGON ((-82.39542 40.96739, -82.39573 40.967..."
3,39,44851,3944851,B5,G6350,S,192283889,1938699,+41.1091751,-082.3915706,N,"MULTIPOLYGON (((-82.25254 41.06463, -82.25341 ..."
4,39,44843,3944843,B5,G6350,S,70792358,259578,+40.6894968,-082.4076022,N,"POLYGON ((-82.42581 40.72716, -82.42518 40.727..."


In [82]:
merged = gpd.sjoin(jobdf, dfzips, how="inner", op="within")

In [83]:
merged.head()

,id,status,noCharge,completedOn,invoice_total,lon,lat,zip,geometry,index_right,...,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10
1,28460822,Completed,False,2019-07-19T19:15:00,0.0,-83.000895,40.019350,43202,POINT (-83.00089 40.01935),1107,...,43202,3943202,B5,G6350,S,6501622,153625,+40.0198116,-083.0151004,N
152,31615020,Completed,False,2019-06-26T16:03:00,240.0,-83.021137,40.030825,43202,POINT (-83.02114 40.03082),1107,...,43202,3943202,B5,G6350,S,6501622,153625,+40.0198116,-083.0151004,N
170,31625407,Completed,False,2019-06-28T22:10:00,6329.0,-82.997172,40.016632,43202,POINT (-82.99717 40.01663),1107,...,43202,3943202,B5,G6350,S,6501622,153625,+40.0198116,-083.0151004,N
336,31652915,Completed,False,2019-06-28T16:13:00,154.0,-83.014002,40.029597,43202,POINT (-83.01400 40.02960),1107,...,43202,3943202,B5,G6350,S,6501622,153625,+40.0198116,-083.0151004,N
558,31716082,Completed,True,2019-07-02T21:10:00,191.0,-83.012113,40.029483,43202,POINT (-83.01211 40.02948),1107,...,43202,3943202,B5,G6350,S,6501622,153625,+40.0198116,-083.0151004,N


In [84]:
f'Percent Match in Zip Codes: {"{:0.1%}".format(sum(merged.zip == merged.ZCTA5CE10)/len(merged))}'

'Percent Match in Zip Codes: 98.6%'

In [85]:
merged[merged.zip != merged.ZCTA5CE10]

,id,status,noCharge,completedOn,invoice_total,lon,lat,zip,geometry,index_right,...,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10
3125,32491302,Completed,False,2019-08-26T19:26:00,0.0,-83.178173,40.147550,43016,POINT (-83.17817 40.14755),633,...,43017,3943017,B5,G6350,S,43714850,1153631,+40.1166209,-083.1304999,N
4799,32932400,Completed,True,2019-09-17T21:13:00,0.0,-83.171566,40.173634,43064,POINT (-83.17157 40.17363),633,...,43017,3943017,B5,G6350,S,43714850,1153631,+40.1166209,-083.1304999,N
7437,32930289,Completed,False,2019-09-18T20:39:00,2462.0,-83.171566,40.173634,43064,POINT (-83.17157 40.17363),633,...,43017,3943017,B5,G6350,S,43714850,1153631,+40.1166209,-083.1304999,N
5853,32333426,Completed,True,2019-08-14T15:00:00,0.0,-83.089044,40.030392,43221,POINT (-83.08904 40.03039),203,...,43220,3943220,B5,G6350,S,17631261,368686,+40.0491532,-083.0742437,N
6045,32373398,Completed,False,2019-08-20T19:51:00,1150.0,-83.089044,40.030392,43221,POINT (-83.08904 40.03039),203,...,43220,3943220,B5,G6350,S,17631261,368686,+40.0491532,-083.0742437,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7369,33072330,Completed,True,2019-09-24T20:14:00,0.0,-83.181413,40.062315,43016,POINT (-83.18141 40.06232),943,...,43002,3943002,B5,G6350,S,2574923,5688,+40.0623618,-083.1708734,N
920,31713030,Completed,False,2019-07-08T13:27:00,0.0,-82.745987,40.231709,43074,POINT (-82.74599 40.23171),675,...,43013,3943013,B5,G6350,S,49783828,111594,+40.2331210,-082.6883233,N
5615,32299845,Completed,False,2019-08-13T22:27:00,1710.0,-82.745987,40.231709,43074,POINT (-82.74599 40.23171),675,...,43013,3943013,B5,G6350,S,49783828,111594,+40.2331210,-082.6883233,N
6072,32380565,Completed,True,2019-08-16T21:11:00,0.0,-82.745987,40.231709,43074,POINT (-82.74599 40.23171),675,...,43013,3943013,B5,G6350,S,49783828,111594,+40.2331210,-082.6883233,N
